# 构建卷积神经网络
- 卷积神经网络的输入层与传统神经网络有些区别，需要重新设计，但是训练模块是基本一致的

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# 首先读取数据
- 分别构建训练接和检测集（验证集）
- DataLoader来迭代取数据

In [11]:
# 定义超参数
input_size = 28 # 图像的总尺寸是28*28
num_classes = 10
num_epochs = 3
batch_size = 64


In [12]:
# 训练集
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data',
                               transform=transforms.ToTensor())

In [13]:
# 构建batch数据
from random import shuffle


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# 卷积网络模块构建
- 一般卷积层，relu层是一起的，两次卷积一次池化
- 注意卷积最后结果还是一个特征图，需要把图转化为向量再做分类或者回归任务

In [19]:
from turtle import forward


class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(  # 输入大小为（1,28,28）
            nn.Conv2d(
                in_channels=1,          # 灰度图
                out_channels=16,        # 要得到多少个特征图
                kernel_size=5,          # 卷积核大小
                stride=1,               # 步长
                padding=2               # 如果希望卷积后的特征图的大小跟原来一样,需要设置padding=(kernel_size-1)/2 if stride=1
            ),                          # 此时输出的特征图为（16，28，28） 
            nn.ReLU(),                  # ReLU层
            nn.MaxPool2d(kernel_size=2),# 进行池化操作（2x2），输出结果为(16,14,14)
        )
        self.conv2 = nn.Sequential(     # 下个一个卷积输入为（16，14，14）
            nn.Conv2d(16,32,5,1,2),     # 输出（32，14，14）
            nn.ReLU(),                  # ReLU层
            nn.MaxPool2d(2)             # 输出（32，7，7）
        )
        self.out = nn.Linear(32*7*7,10) # 全连接层得到的结果
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)        # flatten操作，结果是(batch_size,32x7x7)
        output = self.out(x)
        return output

In [15]:
# 这将准去率作为最终的评估标准
def accuracy(predictions,labels):
    pred = torch.max(predictions.data,1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights,len(labels)

In [24]:
# 训练网络模型
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(),lr=0.01) # 定义优化器

# 开始训练循环
for epoch in range(num_epochs):
    train_rights = []
    for batch_idx,(data,target) in enumerate(train_loader): # 针对容器中的每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output,target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            for (data,target) in test_loader:
                output = net(data)
                right = accuracy(output,target)
                val_rights.append(right)
                
            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
            
            print("当前epoch:{} [{}/{} ({:.0f}%)]\t损失:{:6f}\t训练集准确率:{:.2f}%\t检测集准确率:{:.2f}%".format(
                epoch,batch_idx*batch_size,len(train_loader.dataset),
                100.*batch_idx/len(train_loader),
                loss.data,
                100.*train_r[0].numpy()/train_r[1],
                100.*val_r[0]/val_r[1]
            ))
        

当前epoch:0 [0/60000 (0%)]	损失:2.319847	训练集准确率:9.38%	检测集准确率:9.75%
当前epoch:0 [6400/60000 (11%)]	损失:0.046195	训练集准确率:84.75%	检测集准确率:94.95%
当前epoch:0 [12800/60000 (21%)]	损失:0.069349	训练集准确率:90.33%	检测集准确率:97.08%
当前epoch:0 [19200/60000 (32%)]	损失:0.057271	训练集准确率:92.42%	检测集准确率:96.85%
当前epoch:0 [25600/60000 (43%)]	损失:0.007357	训练集准确率:93.55%	检测集准确率:97.61%
当前epoch:0 [32000/60000 (53%)]	损失:0.029495	训练集准确率:94.38%	检测集准确率:97.92%
当前epoch:0 [38400/60000 (64%)]	损失:0.109703	训练集准确率:94.91%	检测集准确率:97.99%
当前epoch:0 [44800/60000 (75%)]	损失:0.002581	训练集准确率:95.35%	检测集准确率:98.21%
当前epoch:0 [51200/60000 (85%)]	损失:0.113456	训练集准确率:95.66%	检测集准确率:98.14%
当前epoch:0 [57600/60000 (96%)]	损失:0.268721	训练集准确率:95.92%	检测集准确率:98.48%
当前epoch:1 [0/60000 (0%)]	损失:0.065312	训练集准确率:96.88%	检测集准确率:98.50%
当前epoch:1 [6400/60000 (11%)]	损失:0.006033	训练集准确率:98.25%	检测集准确率:98.22%
当前epoch:1 [12800/60000 (21%)]	损失:0.044511	训练集准确率:98.22%	检测集准确率:98.32%
当前epoch:1 [19200/60000 (32%)]	损失:0.241321	训练集准确率:98.15%	检测集准确率:97.96%
当前epoch:1 [25600/60000 (43%)]	损失:0

In [29]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cpu
False


In [28]:
# 训练网络模型
# 实例化
net = CNN()
net.to("cuda")
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(),lr=0.01) # 定义优化器

# 开始训练循环
for epoch in range(num_epochs):
    train_rights = []
    for batch_idx,(data,target) in enumerate(train_loader): # 针对容器中的每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output,target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            for (data,target) in test_loader:
                output = net(data)
                right = accuracy(output,target)
                val_rights.append(right)
                
            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
            
            print("当前epoch:{} [{}/{} ({:.0f}%)]\t损失:{:6f}\t训练集准确率:{:.2f}%\t检测集准确率:{:.2f}%".format(
                epoch,batch_idx*batch_size,len(train_loader.dataset),
                100.*batch_idx/len(train_loader),
                loss.data,
                100.*train_r[0].numpy()/train_r[1],
                100.*val_r[0]/val_r[1]
            ))

AssertionError: Torch not compiled with CUDA enabled